<a href="https://colab.research.google.com/github/armandordorica/MIE1516_A1_Variable_Elimination/blob/master/MIE1516_A1_Q3_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install PrettyTable
!pip install truths

  Created wheel for truths: filename=truths-1.2-cp36-none-any.whl size=7876 sha256=50da255e83eae3c46aea178cf073ef2062757a5be91a0bc73b50c7a555ff2e48
  Stored in directory: /root/.cache/pip/wheels/7b/86/9b/a9954e65607f963fc53e03f67d81950baabdd620685dd1127d
Successfully built truths


In [0]:
import prettytable
import numpy as np
import pandas as pd
pretty=prettytable.PrettyTable()

## Global functions (for now)

In [0]:
def toggle(var):
  if var == 0:
    return 1
  if var == 1:
    return 0

def boolean_truth_table(num_vars):
  truth_table = []
  num_vars = num_vars
  num_cols = num_vars
  var = 1
  num_rows = 2**num_vars
  for col_num in range(0,num_cols):
    truth_table.append([])
    col_num = col_num+1
    # print("col: {} toggle every {} values".format(col_num, 2**(num_vars-col_num)))
    for row_num in range(0, num_rows):
      if row_num%2**(num_vars-col_num)==0:
        var = toggle(var)
      truth_table[col_num-1].append(var)
      # print(truth_table[col_num-1])
      # print(var)
  return truth_table

def generate_truth_table(list_of_variables):
  d = dict()
  num_vars = len(list_of_variables)
  for i in range(0, num_vars): 
    d[str(list_of_variables[i])] = boolean_truth_table(len(list_of_variables))[i]
  df = pd.DataFrame(data=d)
  return df

def multiply_tabular_cpds(tabular_cpd1, tabular_cpd2, var_to_marginalize):
  df1 = tabular_cpd1.to_pandas_df()
  df1 = tabular_cpd1.to_pandas_df()

  vars_factor_1 = tabular_cpd1.get_factor().vars_in_factor()
  vars_factor_1.remove(var_to_marginalize)

  vars_factor_2 = tabular_cpd2.get_factor().vars_in_factor()
  vars_factor_2.remove(var_to_marginalize)

  total_vars = []
  total_vars.append(vars_factor_1[0])
  total_vars.append(vars_factor_2[0])
  output_table = generate_truth_table(total_vars) 

  vars_factor_1.append('Pr')
  vars_factor_2.append('Pr')

  result = pd.merge(df1, df2, on='B', how='inner')
  r0 = result[result['B']==0].Pr_x*result[result['B']==0].Pr_y
  r1 = (result[result['B']==1].Pr_x*result[result['B']==1].Pr_y).reset_index(drop=True)
  output_table['Pr']= r0.add(r1, fill_value=0)
  output_table
  return output_table

In [0]:
class Node:
  def __init__(self, name, parents='', children=''):
    self.name = name
    self.parents = parents
    self.children = children

In [0]:
class MyBayesianModel: 
  def __init__(self, list_of_edges, nodes=[]):
    self.list_of_edges = list(list_of_edges)
    self.nodes = nodes # list(set([k for i in list_of_edges for k in i]))
    self.tabular_cpds=[]



    # class BayesNet():
    # def __init__(self, list_edges):
    #     self.nodes = list(set([k for i in list_edges for k in i]))
    #     self.CPDs = {}
    #     self.parent = {}
    #     self.reset = {'node': self.nodes}

  def get_variables(self):
      list_of_tuples = list(self.list_of_edges)
      list_of_items = [item for t in list_of_tuples for item in t] 
      list_set = set(list_of_items) 
      # convert the set to the list 
      unique_list_of_vars = (list(list_set))
      print(unique_list_of_vars)

  def available_cpds(self):
    for i in range(0, len(self.tabular_cpds)):
      self.tabular_cpds[i].print_factor()

  def eliminate_variable(self, variable_to_eliminate):
    print("Marginalize on {}".format(variable_to_eliminate))
    print("Multiply factors:")
    for i in range(0,len(self.tabular_cpds)):
      if self.tabular_cpds[i].get_factor().contains_var(variable_to_eliminate):
        self.tabular_cpds[i].print_factor()

    print("CPDs are:\n")
    for i in range(0,len(self.tabular_cpds)):
      if self.tabular_cpds[i].get_factor().contains_var(variable_to_eliminate):
        self.tabular_cpds[i].print_cpd()

In [0]:
class Factor:
  def __init__(self, indep_var, dep_vars=[]):
    self.indep_var = indep_var
    self.dep_vars = dep_vars
  
  def print_factor(self):
    if len(self.dep_vars)>0:
      self.dep_vars = set(self.dep_vars)
      self.dep_vars = list(self.dep_vars)
      self.dep_vars.sort()
      #print(self.dep_vars)
      dep_vars = str(self.dep_vars[0])
      for i in range (1, len(self.dep_vars)):
        dep_vars = dep_vars + "," + self.dep_vars[i]
      print("P({}|{})".format(self.indep_var, dep_vars))
    if len(self.dep_vars)==0:
      print("P({})".format(self.indep_var))

  def vars_in_factor(self):
    factors = list()
    factors.append(str(self.indep_var))
    for i in range (0, len(self.dep_vars)):
      factors.append(self.dep_vars[i])
    
    list_set = set(factors) 
    # convert the set to the list 
    unique_list_of_vars = list(list_set)
    unique_list_of_vars.sort(reverse=False)
    # print(unique_list_of_vars)
    return unique_list_of_vars

  def contains_var(self, variable):
    if variable in self.vars_in_factor():
      return True
    else:
      return False

  

In [0]:
class TabularCPD: 
  def __init__(self, variable, variable_card, values, evidence='', evidence_card=''): 
    self.variable = variable 
    self.variable_card = variable_card
    self.values = values
    self.evidence = evidence
    self.evidence_card = evidence_card

  # Initializing factors of the CPD depending on the format (whether evidence is provided or not)
    if len(self.evidence)>0:
      self.factors = []
      self.factors.append(Factor(self.variable, self.evidence))
    
    if len(self.evidence)==0:
      self.factors = []
      self.factors.append(Factor(self.variable))
    
    model.tabular_cpds.append(self)

    edges_with_children=[item for item in model.list_of_edges if item[0] == self.variable]

    if len(edges_with_children)>0: 
      model.nodes.append(Node(self.variable, 
                        self.evidence, 
                        [item for item in model.list_of_edges if item[0] == self.variable][0][1]))
    elif len(edges_with_children)==0: 
      model.nodes.append(Node(self.variable, 
                        self.evidence ))
      
    self.all_variables = []
    self.all_variables.append(self.variable)
    for i in range (0, len(self.evidence)):
      self.all_variables.append(self.evidence[i])

    self.truth_table = generate_truth_table(self.all_variables)
    self.probabilities = []

    for i in range(0, len(self.values)):
      for j in range(0, len(self.values[i])):
        self.probabilities.append(self.values[i][j])

    probs = np.asarray(self.probabilities, dtype=np.float32)
    self.truth_table['Pr'] = probs

  def get_factor(self):
    """Returns all the factors associated with a TabularCPD"""
    return self.factors[0]

  def print_factor(self):
    self.factors[0].print_factor()
  
  def to_truth_table(self):
    # self.all_variables = []
    # self.all_variables.append(self.variable)
    # for i in range (0, len(self.evidence)):
    #   self.all_variables.append(self.evidence[i])
    self.truth_table = generate_truth_table(self.all_variables)

    self.probabilities = []

    for i in range(0, len(self.values)):
      for j in range(0, len(self.values[i])):
        self.probabilities.append(self.values[i][j])

    probs = np.asarray(self.probabilities, dtype=np.float32)
    self.truth_table['Pr'] = probs

    return self.truth_table

  def marginalize(self, var_to_marginalize): 
    #marginalize by "E"
    #var_to_marginalize = 'E'
    #keeping only the values to group by that are not the variable you want to marginalize on 
    group_by_list = list(filter(lambda a: a != var_to_marginalize, self.all_variables))
    df = self.truth_table.groupby(group_by_list).sum()
    df = df.drop(columns=[var_to_marginalize])
    return df 

  def to_pandas_df(self): 
    df = pd.DataFrame.from_records(self.values)
    if self.evidence!='':
      df = df.transpose()

    # for every column append the name of the tabular self variable 
    cols_in_df = len(df.columns)
    rows_in_df = len(df.index)


    for i in range(0, cols_in_df):
      df.rename(columns={ df.columns[i]: str(self.variable)+"_"+str(i) }, inplace = True)

    df.reset_index(inplace=True)

    if self.evidence!='':
      for i in range(0, rows_in_df):
        df.iloc[i,0] = str(self.evidence[0])+"_"+str(i)


    df.set_index('index')

    if self.evidence_card == [2,2]:
      df = pd.DataFrame.from_records(self.values)
      df = df.transpose()

      # for every column append the name of the tabular self variable 
      cols_in_df = len(df.columns)
      rows_in_df = len(df.index)

      for i in range(0, cols_in_df):
        df.rename(columns={ df.columns[i]: str(self.variable)+"_"+str(i) }, inplace = True)

      data = df
      df =generate_truth_table(self.evidence) 
      df['index']= str(self.evidence[0]) + "_" + df[str(self.evidence[0])].astype(str) + "_" + str(self.evidence[1]) + "_"+df[str(self.evidence[1])].astype(str)
      df = df[['index']]
      df_final = pd.concat([df, data], axis=1)
      df_final.set_index('index')
      return df_final.set_index('index')
    return df.set_index('index')

In [0]:
model = MyBayesianModel([('A','C'), ('B', 'C'), ('C', 'G'), ('E', 'D'), ('F', 'D'), ('D', 'G')])

# Defining individual CPDs.

# let 0 denote not wavy and 1  wavy
cpd_a = TabularCPD(variable='A', variable_card = 2, values=[[0.8,0.2]])
cpd_b = TabularCPD(variable='B', variable_card = 2, values=[[0.8,0.2]])
cpd_e = TabularCPD(variable='E', variable_card = 2, values=[[0.8,0.2]])
cpd_f = TabularCPD(variable='F', variable_card = 2, values=[[0.8,0.2]])

cpd_c = TabularCPD(variable='C', variable_card=2, 
                   values=[[0.90, 0.45, 0.45, 0.10], 
                           [0.10, 0.55, 0.55, 0.90]], 
                   evidence=['A', 'B'], 
                   evidence_card=[2,2])

cpd_d = TabularCPD(variable='D', variable_card=2, 
                   values=[[0.90, 0.45, 0.45, 0.10], 
                           [0.10, 0.55, 0.55, 0.90]], 
                   evidence=['E', 'F'], 
                   evidence_card=[2,2])
cpd_g = TabularCPD(variable='G', variable_card=2, 
                   values=[[0.90, 0.45, 0.45, 0.10], 
                           [0.10, 0.55, 0.55, 0.90]], 
                   evidence=['C', 'D'], 
                   evidence_card=[2,2])